In [21]:
file_date = '2021-04-18'

<div style="max-width:1400px;margin-center: auto">
<img src="images\constructor.png" width="600"/>
</div>

In [22]:
import findspark
findspark.init()
import pyspark
from delta import *
from pyspark.sql.types import *
from pyspark.sql.functions import *

#  Create a spark session with Delta
builder = pyspark.sql.SparkSession.builder.appName("constructor") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# Create spark context
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [23]:
from pyspark.sql.functions import col,current_timestamp,lit

In [24]:
constructors_schema = "constructorId INT, constructorRef INT, name STRING, nationality STRING, url STRING "

In [25]:
constructor_df = spark.read.json(f'raw files\\{file_date}\\constructors.json', schema = constructors_schema )
constructor_df.show(2)

+-------------+--------------+----------+-----------+--------------------+
|constructorId|constructorRef|      name|nationality|                 url|
+-------------+--------------+----------+-----------+--------------------+
|            1|          null|   McLaren|    British|http://en.wikiped...|
|            2|          null|BMW Sauber|     German|http://en.wikiped...|
+-------------+--------------+----------+-----------+--------------------+
only showing top 2 rows



In [26]:
# constructor_trans_df = constructor_df.select(col("constructorId").alias("constructor_id"),
#                                              col("constructorRef").alias("constructor_ref"),
#                                              col("name"),
#                                              col("nationality"))
# constructor_trans_df.show(2)

In [27]:
# constructor_drop_df = constructor_df.drop(constructor_df.url)
# constructor_drop_df = constructor_df.drop("url")
# constructor_drop_df = constructor_df.drop(constructor_df["url"])
constructor_drop_df = constructor_df.drop(col("url"))
constructor_drop_df.show(2)

+-------------+--------------+----------+-----------+
|constructorId|constructorRef|      name|nationality|
+-------------+--------------+----------+-----------+
|            1|          null|   McLaren|    British|
|            2|          null|BMW Sauber|     German|
+-------------+--------------+----------+-----------+
only showing top 2 rows



In [28]:
constructor_fnl_df = constructor_drop_df.withColumn("ingestion_date",current_timestamp()).\
                    withColumnRenamed("constructorId","constructor_id").\
                    withColumnRenamed("constructorRef","constructor_ref").\
                    withColumn('file_date',lit(file_date))

In [29]:
constructor_fnl_df.show(2)

+--------------+---------------+----------+-----------+--------------------+----------+
|constructor_id|constructor_ref|      name|nationality|      ingestion_date| file_date|
+--------------+---------------+----------+-----------+--------------------+----------+
|             1|           null|   McLaren|    British|2024-02-11 13:49:...|2021-04-18|
|             2|           null|BMW Sauber|     German|2024-02-11 13:49:...|2021-04-18|
+--------------+---------------+----------+-----------+--------------------+----------+
only showing top 2 rows



In [30]:
constructor_fnl_df.write.mode('overwrite').format('delta').saveAsTable('default.constructors')